In [ ]:
site = "m1s1"
neuron_number = 238
model_name = "rf_cnn"

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from torch.utils.data import Dataset,DataLoader
from training.utils import *

from modeling import *

val_img = np.load(f'/val_img_{site}.npy')
val_resp = np.load(f'/valRsp_{site}.npy')
val_img = np.reshape(val_img,(-1,1,50,50))

net = model_rf_cnn_one_neuron()

In [ ]:
net.load_state_dict(torch.load(f'../results/{site}_model_{model_name}/neuron_{neuron_number}.pth', map_location=torch.device('cpu')))
corr = pc_one_neuron(net,neuron_number,val_img,val_resp)
with torch.no_grad():
    net.eval()
    pred = net(torch.from_numpy(np.float32(val_img))).detach().numpy().T[0]
    real = val_resp[:,neuron_number]
    zipped = list(zip(list(range(1000)),pred,real))
    zipped.sort(key=lambda x:x[2], reverse=True)
plt.plot(list(range(1000)),[x[2] for x in zipped], label = "Real",linewidth=1.5)
plt.plot(list(range(1000)),[x[1] for x in zipped], label = "Prediction",linewidth=2.5)
plt.xlabel("Image",fontsize=20)
plt.ylabel("Response (dF/F)",fontsize=20)
plt.title(f'{dname}',fontsize=26,fontfamily='monospace')
plt.annotate(f'CORR. = {corr:.3f}',xy=(0.6, 0.6),xycoords='figure fraction',fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)


plt.tight_layout()
plt.legend(fontsize=20)
plt.savefig(f'z_tc{model_name}.pdf', format='pdf')